In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
#Getting the fundamentals data
df = pd.read_csv("SF0_20170108.csv",header=None)
df.columns = ['label','date','value']

#wee bit of cleanup on the labels
dfTemp = pd.DataFrame(df['label'].str.split('_').tolist(),columns=['ticker','attributes','MRY'])
df['ticker'],df['attributes'] = dfTemp['ticker'],dfTemp['attributes']
df = df.drop("label",axis=1)
df

,date,value,ticker,attributes
0,2013-12-31,0.000000e+00,AAC,ACCOCI
1,2014-12-31,0.000000e+00,AAC,ACCOCI
2,2015-12-31,0.000000e+00,AAC,ACCOCI
3,2013-12-31,2.952900e+07,AAC,ASSETSC
4,2014-12-31,7.992200e+07,AAC,ASSETSC
5,2015-12-31,8.652400e+07,AAC,ASSETSC
6,2013-12-31,5.210900e+07,AAC,ASSETSNC
7,2014-12-31,6.603000e+07,AAC,ASSETSNC
8,2015-12-31,2.295250e+08,AAC,ASSETSNC
9,2013-12-31,8.163800e+07,AAC,ASSETS


In [3]:
#extracting the months and years out of dates and dropping the latter
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df = df.drop('date',axis=1)

#loading up the historical price data
prices = pd.read_csv('WIKI_PRICES_212b326a081eacca455e13140d7bb9db.csv',usecols=['ticker','date','adj_close'])
prices['date'] = pd.to_datetime(prices['date'])
prices['year'] = prices['date'].dt.year
prices['month'] = prices['date'].dt.month

In [11]:
#prices = prices.set_index('date')
prices_shrt = prices[prices['year']>2010]
prices_shrt['value'] = 1.0
prices_shrt['value'] = prices_shrt.groupby(['ticker', 'year','month']).pct_change()
prices_shrt['value'] = prices_shrt['value'] + 1

yearlyR = prices_shrt.groupby(by=['ticker','year'],as_index=False)['value'].prod()
yearlyR['value'] = yearlyR['value'] -1
yearlyR['attributes'] = 'yearlyReturn'

FiveYrR = prices_shrt.groupby(by=['ticker'],as_index=False)['value'].prod()
FiveYrR['value'] = FiveYrR['value'] -1
FiveYrR['attributes'] = 'FiveYrReturn'

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [12]:
print(yearlyR)
print(FiveYrR)

      ticker  year  value    attributes
0          A  2011    0.0  yearlyReturn
1          A  2012    0.0  yearlyReturn
2          A  2013    0.0  yearlyReturn
3          A  2014    0.0  yearlyReturn
4          A  2015    0.0  yearlyReturn
5          A  2016    0.0  yearlyReturn
6          A  2017    0.0  yearlyReturn
7         AA  2016    0.0  yearlyReturn
8         AA  2017    0.0  yearlyReturn
9        AAL  2011    0.0  yearlyReturn
10       AAL  2012    0.0  yearlyReturn
11       AAL  2013    0.0  yearlyReturn
12       AAL  2014    0.0  yearlyReturn
13       AAL  2015    0.0  yearlyReturn
14       AAL  2016    0.0  yearlyReturn
15       AAL  2017    0.0  yearlyReturn
16      AAMC  2012    0.0  yearlyReturn
17      AAMC  2013    0.0  yearlyReturn
18      AAMC  2014    0.0  yearlyReturn
19      AAMC  2015    0.0  yearlyReturn
20      AAMC  2016    0.0  yearlyReturn
21      AAMC  2017    0.0  yearlyReturn
22       AAN  2011    0.0  yearlyReturn
23       AAN  2012    0.0  yearlyReturn


In [20]:
#averaging the prices of the fiscal year-end month
avgP = prices.groupby(['ticker','year','month'],as_index=False).mean()

#Calculating returns per ticker
prices = prices.set_index('date')
prices_shrt = prices[prices['year']>2011]
prices_shrt['value'] = 1.0
prices_shrt['value'] = prices_shrt.groupby(['ticker', 'year','month']).pct_change()
prices_shrt['value'] = prices_shrt['value'] + 1
yearlyR = prices_shrt.groupby(by=['ticker','year'],as_index=False)['value'].prod()
yearlyR['value'] = yearlyR['value'] -1
yearlyR['attributes'] = 'yearlyReturn'
df = df.append(yearlyR)


#average year-end prices
fuit = df.groupby(['ticker','year','month'],as_index=False).count()
fuit['attributes'] = 'price'
fuit = pd.merge(fuit,avgP, on = ['ticker','year','month'], how='left')
fuit = fuit.drop(['value'],axis=1)
fuit.columns = ['ticker','year','month','attributes','value']

#appending the calculated data to the master dataframe
df = df.append(fuit)

#drop the NaN
df = df[np.isfinite(df['value'])]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [21]:
#objective
#Predict 5 year returns with fundamentals
## Find the factors that contribute to those return
## Which ex ante valuation metrics are good proxies to predict 5y growth?
#### TO DO
#### Calculate 5y growth rate per ticker
#### Calculate ex ante metrics (beginning of observation period)

## Which periodic metric is the most strongly linked to the ending YoY growth?
#### TO DO
#### Calculate 5y growth rate per ticker
#### Calculate average metrics for the period



#INDUSTRY EXTRACTOR
#https://mktstk.com/2015/03/03/sic-lookup-by-stock-symbol/

#df.to_csv("fuit.csv")
pivoted['DEBT']

ticker  year
A       2009    2.905000e+09
        2010    3.691000e+09
        2011    2.185000e+09
        2012    2.362000e+09
        2013    2.699000e+09
        2014    1.663000e+09
        2015    1.655000e+09
        2016    1.912000e+09
        2017             NaN
AA      2016             NaN
        2017             NaN
AAC     2012             NaN
        2013    4.307500e+07
        2014    2.864100e+07
        2015    1.451410e+08
AAL     2009    1.169700e+10
        2010    1.113600e+10
        2011    8.220000e+09
        2012    8.535000e+09
        2013    1.679900e+10
        2014    1.772000e+10
        2015    2.056100e+10
        2016             NaN
        2017             NaN
AAMC    2012             NaN
        2013             NaN
        2014             NaN
        2015             NaN
        2016             NaN
        2017             NaN
                    ...     
ZNGA    2012    1.000000e+08
        2013    0.000000e+00
        2014    0.000000e+00
 

In [22]:
df['attributes'].unique()

array(['ACCOCI', 'ASSETSC', 'ASSETSNC', 'ASSETS', 'BVPS', 'CAPEX',
       'CASHNEQUSD', 'CASHNEQ', 'COR', 'CURRENTRATIO', 'DEBTUSD', 'DEBT',
       'DEPAMOR', 'DE', 'DILUTIONRATIO', 'DPS', 'EBITDAUSD', 'EBITDA',
       'EBITUSD', 'EBIT', 'EBT', 'EPSDIL', 'EPSUSD', 'EPS', 'EQUITYUSD',
       'EQUITY', 'FCFPS', 'FCF', 'GP', 'INTANGIBLES', 'INTEXP', 'INVCAP',
       'INVENTORY', 'LIABILITIESC', 'LIABILITIESNC', 'LIABILITIES',
       'NCFCOMMON', 'NCFDEBT', 'NCFDIV', 'NCFF', 'NCFI', 'NCFO', 'NCFX',
       'NCF', 'NETINCCMNUSD', 'NETINCCMN', 'NETINCDIS', 'NETINC',
       'PAYABLES', 'PB', 'PREFDIVIS', 'RECEIVABLES', 'RETEARN',
       'REVENUEUSD', 'REVENUE', 'RND', 'SGNA', 'SHARESWADIL', 'SHARESWA',
       'TANGIBLES', 'TAXEXP', 'TBVPS', 'WORKINGCAPITAL', 'yearlyReturn',
       'price'], dtype=object)

In [23]:
results = pd.DataFrame()
pivoted = pd.pivot_table(df, values='value', index=['ticker', 'year'],columns='attributes')
pivoted

attributes         ACCOCI        ASSETS       ASSETSC      ASSETSNC  \
ticker year                                                           
A      2009 -1.850000e+08  7.612000e+09  3.961000e+09  3.651000e+09   
       2010 -8.800000e+07  9.696000e+09  6.169000e+09  3.527000e+09   
       2011  1.160000e+08  9.057000e+09  5.569000e+09  3.488000e+09   
       2012 -1.110000e+08  1.053600e+10  4.629000e+09  5.907000e+09   
       2013  9.100000e+07  1.068600e+10  4.983000e+09  5.703000e+09   
       2014 -3.340000e+08  1.081500e+10  5.509000e+09  5.306000e+09   
       2015 -3.910000e+08  7.479000e+09  3.686000e+09  3.793000e+09   
       2016 -5.030000e+08  7.802000e+09  3.635000e+09  4.167000e+09   
       2017           NaN           NaN           NaN           NaN   
AA     2016           NaN           NaN           NaN           NaN   
       2017           NaN           NaN           NaN           NaN   
AAC    2012           NaN           NaN           NaN           NaN   
       2013  0.000000e+00  8.163800e+07  2.952900e+07  5.210900e+07   
       2014  0.000000e+00  1.459520e+08  7.992200e+07  6.603000e+07   
       2015  0.000000e+00  3.160490e+08  8.652400e+07  2.295250e+08   
AAL    2009 -2.724000e+09  2.543800e+10  6.642000e+09  1.879600e+10   
       2010 -2.755000e+09  2.508800e+10  6.838000e+09  1.825000e+10   
       2011 -3.964000e+09  2.384800e+10  6.757000e+09  1.709100e+10   
       2012 -2.980000e+09  2.351000e+10  7.072000e+09  1.643800e+10   
       2013 -2.032000e+09  4.227800e+10  1.432300e+10  2.795500e+10   
       2014 -4.559000e+09  4.322500e+10  1.175000e+10  3.147500e+10   
       2015 -4.732000e+09  4.841500e+10  9.985000e+09  3.843000e+10   
       2016           NaN           NaN           NaN           NaN   
       2017           NaN           NaN           NaN           NaN   
AAMC   2012           NaN           NaN           NaN           NaN   
       2013           NaN           NaN           NaN           NaN   
       2014           NaN           NaN           NaN           NaN   
       2015           NaN           NaN           NaN           NaN   
       2016           NaN           NaN           NaN           NaN   
       2017           NaN           NaN           NaN           NaN   
...                   ...           ...           ...           ...   
ZNGA   2012 -1.447000e+06  2.576320e+09  1.484370e+09  1.091950e+09   
       2013 -1.046000e+06  2.279085e+09  1.241399e+09  1.037686e+09   
       2014 -2.917500e+07  2.348793e+09  1.082939e+09  1.265854e+09   
       2015 -5.238800e+07  2.124630e+09  1.112290e+09  1.012340e+09   
       2016           NaN           NaN           NaN           NaN   
       2017           NaN           NaN           NaN           NaN   
ZOES   2014           NaN           NaN           NaN           NaN   
       2015           NaN           NaN           NaN           NaN   
       2016           NaN           NaN           NaN           NaN   
       2017           NaN           NaN           NaN           NaN   
ZQK    2012           NaN           NaN           NaN           NaN   
       2013           NaN           NaN           NaN           NaN   
       2014           NaN           NaN           NaN           NaN   
       2015           NaN           NaN           NaN           NaN   
ZTS    2010           NaN           NaN           NaN           NaN   
       2011 -6.500000e+07  5.711000e+09  2.311000e+09  3.400000e+09   
       2012 -1.570000e+08  6.262000e+09  2.864000e+09  3.398000e+09   
       2013 -2.190000e+08  6.558000e+09  3.357000e+09  3.201000e+09   
       2014 -3.610000e+08  6.588000e+09  3.465000e+09  3.123000e+09   
       2015 -6.220000e+08  7.913000e+09  3.830000e+09  4.083000e+09   
       2016           NaN           NaN           NaN           NaN   
       2017           NaN           NaN           NaN           NaN   
ZUMZ   2010  1.010000e+05  2.602650e+08  1.737140e+08  8.655100e+07   
       2011 -1.700000e+0

In [24]:
print(pivoted['INVCAP'])
print((pivoted['EQUITY']+pivoted['DEBT']-pivoted['CASHNEQ']))

ticker  year
A       2009    4.527000e+09
        2010    4.155000e+09
        2011    3.882000e+09
        2012    4.543000e+09
        2013    5.145000e+09
        2014    5.412000e+09
        2015    3.102000e+09
        2016    3.555000e+09
        2017             NaN
AA      2016             NaN
        2017             NaN
AAC     2012             NaN
        2013    8.003300e+07
        2014    9.364700e+07
        2015    2.899110e+08
AAL     2009    2.879400e+10
        2010    2.682600e+10
        2011    2.241700e+10
        2012    2.170400e+10
        2013    3.669900e+10
        2014    3.944200e+10
        2015    4.794600e+10
        2016             NaN
        2017             NaN
AAMC    2012             NaN
        2013             NaN
        2014             NaN
        2015             NaN
        2016             NaN
        2017             NaN
                    ...     
ZNGA    2012    1.510456e+09
        2013    1.287296e+09
        2014    1.082766e+09
 

In [25]:

#relevant ratios

#financial performance
results['ROE'] = pivoted['NETINC']/pivoted['EQUITY']
results['ROA'] = pivoted['EBITDA']*(1-pivoted['TAXEXP']/pivoted['EBT'])/pivoted['ASSETS']
results['ROIC'] = (pivoted['EBIT']*(1-pivoted['TAXEXP']/pivoted['EBT']))/(pivoted['EQUITY']+pivoted['DEBT']-pivoted['CASHNEQ'])
results['FCFEtoEq'] = (pivoted['NCFO']-pivoted['CAPEX'])/pivoted['EQUITY']
results['FCFFtoOpAss'] = (pivoted['NCFO']+pivoted['INTEXP']*(1-pivoted['TAXEXP']/pivoted['EBT'])-pivoted['CAPEX'])/pivoted['EQUITY']

#credibility
####results['NOA'] = pivoted['ASSETS']-pivoted['CASHNEQ']-(pivoted[''])
####results['CashAcc'] = pivoted['REVENUE']/pivoted['ASSETS']
####results['AssAcc'] = pivoted['REVENUE']/pivoted['ASSETS']

#Valuation
results['PEdil'] = pivoted['price']/(pivoted['NETINC']/pivoted['SHARESWADIL'])
results['PtoCF'] = pivoted['price']/(pivoted['NCFO']/pivoted['SHARESWADIL'])
results['PtoBV'] = pivoted['price']/pivoted['BVPS']
results['PtoFCF'] = pivoted['price']/pivoted['FCFPS']
results['mktCap'] = pivoted['price']*pivoted['SHARESWA']


#debt
results['DE'] = pivoted['DEBT']/pivoted['EQUITY']
results['Leverage'] = pivoted['ASSETS']/pivoted['EQUITY']

#operational efficiency
results['CashConvCycle'] = 365/(pivoted['REVENUE']/pivoted['RECEIVABLES'])+365/(pivoted['COR']/pivoted['INVENTORY'])-365/(pivoted['COR']/pivoted['PAYABLES'])
results['AssTurnover'] = pivoted['REVENUE']/pivoted['ASSETS']
results['returns'] = pivoted['yearlyReturn']

results.to_csv('results.csv')

In [26]:
#metrix['ROE'] = df['value'].loc[(df['attributes']=='NETINCCMN') & (df['year']==2015)]
results

ROE       ROA       ROIC  FCFEtoEq  FCFFtoOpAss       PEdil  \
ticker year                                                                     
A      2009 -0.012370 -0.149520  -0.307990  0.213488     0.057975 -283.089360   
       2010  0.211896  0.100924   0.286357  0.257745     0.287141   16.729972   
       2011  0.234912  0.148441   0.369634  0.331941     0.351517   11.396202   
       2012  0.222501  0.151613   0.243530  0.274411     0.295958   10.953800   
       2013  0.138857  0.109712   0.156672  0.254446     0.272972   23.352173   
       2014  0.103565  0.096691   0.138981  0.173929     0.194794   32.565995   
       2015  0.096232  0.092226   0.128807  0.143509     0.157846   29.707520   
       2016  0.108885  0.093831   0.135320  0.219656     0.234067   32.325656   
       2017       NaN       NaN        NaN       NaN          NaN         NaN   
AA     2016       NaN       NaN        NaN       NaN          NaN         NaN   
       2017       NaN       NaN        NaN       NaN          NaN         NaN   
AAC    2012       NaN       NaN        NaN       NaN          NaN         NaN   
       2013  0.096053  0.039817   0.031796  2.006355     2.101653         NaN   
       2014  0.077436  0.085162   0.115328  0.242342     0.256690         NaN   
       2015  0.078882  0.060716   0.051112  0.407458     0.425292         NaN   
AAL    2009  0.420751  0.003162  -0.111205 -0.680711    -0.859386         NaN   
       2010  0.119392  0.052315   0.044892 -0.810900    -1.005088         NaN   
       2011  0.278301 -0.003438 -13.272727 -0.331458    -0.445507         NaN   
       2012  0.234882 -0.026044   1.778873 -0.381871    -0.442584         NaN   
       2013  0.671549 -0.006049  -0.093657 -1.340535    -1.604225         NaN   
       2014  1.426027  0.116493   0.204633  4.135576     4.529376         NaN   
       2015  1.350488  0.241937   0.360829  2.194321     2.451780         NaN   
       2016       NaN       NaN        NaN       NaN          NaN         NaN   
       2017       NaN       NaN        NaN       NaN          NaN         NaN   
AAMC   2012       NaN       NaN        NaN       NaN          NaN         NaN   
       2013       NaN       NaN        NaN       NaN          NaN         NaN   
       2014       NaN       NaN        NaN       NaN          NaN         NaN   
       2015       NaN       NaN        NaN       NaN          NaN         NaN   
       2016       NaN       NaN        NaN       NaN          NaN         NaN   
       2017       NaN       NaN        NaN       NaN          NaN         NaN   
...               ...       ...        ...       ...          ...         ...   
ZNGA   2012 -0.114734 -0.009219  -0.138579  0.288973     0.288973   -8.542461   
       2013 -0.019700  0.016054  -0.026261  0.017808     0.017808  -87.700911   
       2014 -0.119165 -0.061994  -0.131617 -0.000195    -0.000195   -9.911346   
       2015 -0.068000 -0.033330  -0.116446 -0.020946    -0.020946  -19.616829   
       2016       NaN       NaN        NaN       NaN          NaN         NaN   
       2017       NaN       NaN        NaN       NaN          NaN         NaN   
ZOES   2014       NaN       NaN        NaN       NaN          NaN         NaN   
       2015       NaN       NaN        NaN       NaN          NaN         NaN   
       2016       NaN       NaN        NaN       NaN          NaN         NaN   
       2017       NaN       NaN        NaN       NaN          NaN         NaN   
ZQK    2012       NaN       NaN        NaN       NaN          NaN         NaN   
       2013       NaN       NaN        NaN       NaN          NaN         NaN   
       2014       NaN       NaN        NaN       NaN          NaN         NaN   
       2015       NaN       NaN        NaN       NaN          NaN         NaN   
ZTS    2010       NaN       NaN        NaN       NaN          NaN         NaN   
       2011  0.065860  0.065392   0.058112  0.169892     0.169892         NaN   
       2012  0.108296  0.089239   0.101608  0.144